# 

In [2]:
#importing
import csv
import psycopg2
import matplotlib.pyplot as plt

In [3]:
conn = psycopg2.connect(host='localhost', database='seasonal_consumption', user='dap', password='dap')
cur = conn.cursor()

In [18]:
cur.execute("""select count(*) from electric_consumption;""")
count_elec= cur.fetchall()
cur.execute("""select count(*) from water_consumption;""") 
coun_water = cur.fetchall()
cur.execute("""select count(*) from newyork_weather;""")
coun_weather = cur.fetchall()

conn.commit()



In [19]:
#checking counts of 3 table
print(count_elec,coun_water,coun_weather)

[(274489,)] [(41665,)] [(2922,)]


In [22]:
#checking for possiblities of cartesian product in joining in table Electrical_consumption
cur.execute("""SELECT
                Account_Name,
                "location",
                Revenue_DATE,
                Meter_Number,
                BILL_ID,
                COUNT(*)
                FROM (SELECT 
                    ELEC.BILL_ID,
                    ELEC.Development_Name, 
                    ELEC.Borough, 
                    ELEC.Account_Name,
                    ELEC.location ,
                    ELEC.Revenue_DATE, 
                    ELEC.Service_Start_Date, 
                    ELEC.Service_End_Date, 
                    ELEC.Days, 
                    ELEC.Meter_Number, 
                    SUM(ELEC.Consumption_KWH), 
                    SUM(ELEC.KWH_Charges), 
                    SUM(ELEC.Consumption_KW), 
                    SUM(ELEC.KW_Charges)
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) ELEC
                GROUP BY 1,2,3,4,5
                HAVING COUNT(*) >1;
                """)

result_elec=cur.fetchall()
conn.commit()

result_elec

[('RED HOOK EAST',
  'BLD 29 - PLP',
  datetime.datetime(2020, 4, 1, 0, 0),
  '009956005',
  '8885952',
  2)]

In [25]:
#we got the below 2 records from above observation which can cause to cartesian product
cur.execute("""select * from electric_consumption ELEC
                where 1=1
                AND TRIM(Account_Name) ='RED HOOK EAST'
                AND TRIM(ELEC.location)= 'BLD 29 - PLP'
                AND Meter_Number='009956005'
                AND BILL_ID= '8885952';
                """)

result_elec_d=cur.fetchall()
conn.commit()

result_elec_d

[(227315,
  '8885952',
  'RED HOOK EAST',
  'BROOKLYN',
  'RED HOOK EAST',
  'BLD 29 - PLP',
  datetime.datetime(2020, 4, 1, 0, 0),
  datetime.datetime(2020, 4, 9, 0, 0),
  datetime.datetime(2020, 5, 6, 0, 0),
  27,
  '009956005',
  Decimal('3240.0'),
  Decimal('224.9'),
  Decimal('5.0'),
  Decimal('0.0'),
  'EL2'),
 (265105,
  '8885952',
  'RED HOOK EAST',
  'BROOKLYN',
  'RED HOOK EAST',
  'BLD 29 - PLP',
  datetime.datetime(2020, 4, 1, 0, 0),
  datetime.datetime(2020, 4, 9, 0, 0),
  datetime.datetime(2020, 5, 8, 0, 0),
  29,
  '009956005',
  Decimal('3240.0'),
  Decimal('224.9'),
  Decimal('5.0'),
  Decimal('0.0'),
  'EL2')]

In [30]:
#As the bill_id and consumption of electricity of 2 records are same but the service_end_date & days is different, 
#it is possible other record is duplicated
#we will need to delete the one record to avoid the cross join
try:
    
    cur.execute("""DELETE FROM electric_consumption ELEC
        where Account_Name ='RED HOOK EAST'
        AND TRIM(ELEC.location)= 'BLD 29 - PLP'
        AND Meter_Number='009956005'
        AND BILL_ID= '8885952'
        AND days=27
        """)
    conn.commit()
    print("Record Deleted")
    
except Exception as e:
    print(f"Error: {e}")
    

Record Deleted


In [ ]:
#changed the column datatype

cur.execute("""ALTER TABLE water_consumption ALTER COLUMN Consumption_hcf  TYPE integer USING (Consumption_hcf::integer);""")
conn.commit()

In [32]:
# checking it in table Water_Consumption

#ALTER TABLE water_consumption ALTER COLUMN Consumption_hcf  TYPE integer USING (Consumption_hcf::integer);

cur.execute("""
    SELECT
    WAT.BILL_ID,
    WAT.Account_Name,
    WAT.location ,
    WAT.Revenue_DATE,
    WAT.Meter_Number,
    COUNT(*)
        FROM (SELECT 
                WAT.BILL_ID,
                WAT.Development_Name, 
                WAT.Borough, 
                WAT.Account_Name,
                WAT.location,
                WAT.Revenue_DATE, 
                WAT.Service_Start_Date, 
                WAT.Service_End_Date, 
                WAT.Days, 
                WAT.Meter_Number, 
                SUM(WAT.Consumption_hcf)
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8,9,10)WAT
        GROUP BY 1,2,3,4,5
    HAVING COUNT(*) >1;
    """)
result_water=cur.fetchall()
conn.commit()

result_water

[('8191723',
  'FHA REPOSSESSED HOUSES (GROUP IV)',
  '111-20 205TH STREET',
  datetime.datetime(2019, 9, 1, 0, 0),
  'V48022226',
  2),
 ('8190854',
  'SOUTH JAMAICA II',
  'BLD 17',
  datetime.datetime(2019, 9, 1, 0, 0),
  'G13008478',
  2)]

In [33]:
#we got the below 4 records from above observation which can cause to cartesian product
cur.execute("""select * from water_consumption WAT
            WHERE TRIM(Account_Name) = 'FHA REPOSSESSED HOUSES (GROUP IV)'
            AND TRIM(WAT.location) ='111-20 205TH STREET'
            AND BILL_ID ='8191723'
            AND Meter_Number = 'V48022226'
            """)
result_water_d=cur.fetchall()
conn.commit()

result_water_d

[(4352,
  '8191723',
  'FHA REPOSSESSED HOUSES (GROUP IV)',
  'FHA',
  'FHA REPOSSESSED HOUSES (GROUP IV)',
  '111-20 205TH STREET',
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2019, 8, 22, 0, 0),
  datetime.datetime(2019, 9, 25, 0, 0),
  34,
  'V48022226',
  100,
  'Basic Water and Sewer'),
 (19897,
  '8191723',
  'FHA REPOSSESSED HOUSES (GROUP IV)',
  'FHA',
  'FHA REPOSSESSED HOUSES (GROUP IV)',
  '111-20 205TH STREET',
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2019, 8, 22, 0, 0),
  datetime.datetime(2019, 9, 22, 0, 0),
  31,
  'V48022226',
  100,
  'Basic Water and Sewer')]

In [34]:
#As the bill_id and consumption of water of above 2 records are same but the service_end_date & days is different, 
#it is possible other record is duplicated
#we will need to delete the one record to avoid the cross join
try:
    
    cur.execute("""DELETE FROM water_consumption WAT
        WHERE TRIM(Account_Name) = 'FHA REPOSSESSED HOUSES (GROUP IV)'
        AND TRIM(WAT.location) ='111-20 205TH STREET'
        AND BILL_ID ='8191723'
        AND Meter_Number = 'V48022226'
        AND days=34
        """)
    conn.commit()
    print("Record Deleted")
    
except Exception as e:
    print(f"Error: {e}")
    

Record Deleted


In [35]:
#we got the below 4 records from above observation which can cause to cartesian product
cur.execute("""select * from water_consumption WAT
            WHERE TRIM(Account_Name) = 'SOUTH JAMAICA II'
            AND TRIM(WAT.location) ='BLD 17'
            AND BILL_ID ='8190854'
            AND Meter_Number = 'G13008478'
            """)
result_water_d2=cur.fetchall()
conn.commit()

result_water_d2

[(8184,
  '8190854',
  'SOUTH JAMAICA II',
  'QUEENS',
  'SOUTH JAMAICA II',
  'BLD 17',
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2019, 8, 22, 0, 0),
  datetime.datetime(2019, 9, 22, 0, 0),
  31,
  'G13008478',
  53,
  'Basic Water and Sewer'),
 (36823,
  '8190854',
  'SOUTH JAMAICA II',
  'QUEENS',
  'SOUTH JAMAICA II',
  'BLD 17',
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2010, 8, 22, 0, 0),
  datetime.datetime(2019, 9, 22, 0, 0),
  3318,
  'G13008478',
  0,
  'Basic Water and Sewer')]

In [36]:
#As the bill_id of above 2 records are same and consumption of water is 0 
#it is possible other record is wrongly entered 
#we will need to delete the one record to avoid the cross join
try:
    
    cur.execute("""DELETE FROM water_consumption WAT
        WHERE TRIM(Account_Name) = 'SOUTH JAMAICA II'
        AND TRIM(WAT.location) ='BLD 17'
        AND BILL_ID ='8190854'
        AND Meter_Number = 'G13008478'
        AND days=3318
        """)
    conn.commit()
    print("Record Deleted")
    
except Exception as e:
    print(f"Error: {e}")
    

Record Deleted


In [60]:

try:
    
    cur.execute('''SELECT
                ELEC.Development_Name, 
                ELEC.Borough, 
                ELEC.Account_Name,
                ELEC.location ,
                ELEC.Revenue_DATE, 
                ELEC.Service_Start_Date, 
                ELEC.Service_End_Date, 
                ELEC.Days, 
                ELEC.Consumption_KWH, 
                ELEC.KWH_Charges, 
                ELEC.Consumption_KW, 
                ELEC.KW_Charges, 
                WAT.Consumption_hcf AS Consumption_hcf
                FROM(SELECT
                    ELEC.BILL_ID AS BILL_ID,
                    ELEC.Development_Name AS Development_Name, 
                    ELEC.Borough AS Borough, 
                    ELEC.Account_Name AS Account_Name,
                    ELEC.location AS "location" ,
                    ELEC.Revenue_DATE AS Revenue_DATE, 
                    ELEC.Service_Start_Date AS Service_Start_Date, 
                    ELEC.Service_End_Date AS Service_End_Date, 
                    ELEC.Days AS Days, 
                    ELEC.Meter_Number AS Meter_Number, 
                    SUM(ELEC.Consumption_KWH) AS Consumption_KWH, 
                    SUM(ELEC.KWH_Charges) AS KWH_Charges, 
                    SUM(ELEC.Consumption_KW) AS Consumption_KW, 
                    SUM(ELEC.KW_Charges) AS KW_Charges
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) ELEC
                Left JOIN (SELECT 
                        WAT.BILL_ID,
                        WAT.Development_Name, 
                        WAT.Borough, 
                        WAT.Account_Name,
                        WAT.location ,
                        WAT.Revenue_DATE, 
                        WAT.Service_Start_Date, 
                        WAT.Service_End_Date, 
                        WAT.Days, 
                        WAT.Meter_Number, 
                        SUM(WAT.Consumption_hcf) Consumption_hcf
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) WAT
                ON ELEC.revenue_date= WAT.revenue_date
                AND ELEC.location=WAT.location
                AND ELEC.Account_Name=WAT.Account_Name;
                ''')
    result_join=cur.fetchall()
    conn.commit()
except psycopg2.errors.InFailedSqlTransaction as e:
    conn.rollback()
    print("Transaction rolled back:", e)
except Exception as e:
    # handle any other exceptions that might occur
    print("Error occurred:", e)

In [59]:
print(result_join[1:10])
print("Count of joining to tables",len(result_join))

[('FHA REPOSSESSED HOUSES (GROUP IX)', 'FHA', 'FHA REPOSSESSED HOUSES (GROUP IX)', '349 BERRIMAN STREET', datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2012, 12, 24, 0, 0), datetime.datetime(2013, 1, 24, 0, 0), 31, Decimal('67.0'), Decimal('5.2'), Decimal('0.0'), Decimal('0.0'), 12), ('FHA REPOSSESSED HOUSES (GROUP IX)', 'FHA', 'FHA REPOSSESSED HOUSES (GROUP IX)', '359 BERRIMAN STREET', datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2012, 12, 24, 0, 0), datetime.datetime(2013, 1, 24, 0, 0), 31, Decimal('20.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), 13), ('BOYNTON AVENUE REHAB', 'BRONX', 'BOYNTON AVENUE REHAB', 'BLD 01', datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2012, 12, 24, 0, 0), datetime.datetime(2013, 1, 24, 0, 0), 31, Decimal('3132.0'), Decimal('181.8'), Decimal('5.4'), Decimal('67.5'), 767), ('CONLON LIHFE TOWER', 'QUEENS', 'CONLON LIHFE TOWER', 'BLD 01', datetime.datetime(2013, 1, 1, 0, 0), datetime.datetime(2012, 12, 24, 0, 0), datetime.d

In [25]:
#CHECKING OF GETTING CARTESIAN PRODUCT AFTER LEFT JOIN

try:
    
    cur.execute('''SELECT
    Development_Name,
    Borough,
    Account_Name,
    location,
    Revenue_DATE,
    COUNT(*)
    FROM (
    SELECT
                ELEC.Development_Name, 
                ELEC.Borough, 
                ELEC.Account_Name,
                ELEC.location ,
                ELEC.Revenue_DATE, 
                ELEC.Service_Start_Date elec_Service_Start_Date, 
                ELEC.Service_End_Date elec_Service_End_Date, 
                ELEC.Days elec_Days, 
                WAT.Service_Start_Date water_Service_Start_Date, 
                WAT.Service_End_Date water_Service_End_Date, 
                WAT.Days Water_days, 
                SUM(ELEC.Consumption_KWH) elec_Consumption_KWH, 
                SUM(ELEC.KWH_Charges) elec_KWH_Charges, 
                SUM(ELEC.Consumption_KW) elec_Consumption_KW, 
                SUM(ELEC.KW_Charges) elec_KW_Charges, 
                SUM(WAT.Consumption_hcf) AS water_Consumption_hcf
                FROM(SELECT
                    ELEC.BILL_ID AS BILL_ID,
                    ELEC.Development_Name AS Development_Name, 
                    ELEC.Borough AS Borough, 
                    ELEC.Account_Name AS Account_Name,
                    ELEC.location AS "location" ,
                    ELEC.Revenue_DATE AS Revenue_DATE, 
                    ELEC.Service_Start_Date AS Service_Start_Date, 
                    ELEC.Service_End_Date AS Service_End_Date, 
                    ELEC.Days AS Days, 
                    ELEC.Meter_Number AS Meter_Number, 
                    SUM(ELEC.Consumption_KWH) AS Consumption_KWH, 
                    SUM(ELEC.KWH_Charges) AS KWH_Charges, 
                    SUM(ELEC.Consumption_KW) AS Consumption_KW, 
                    SUM(ELEC.KW_Charges) AS KW_Charges
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) ELEC
                Left JOIN (SELECT 
                        WAT.BILL_ID,
                        WAT.Development_Name, 
                        WAT.Borough, 
                        WAT.Account_Name,
                        WAT.location ,
                        WAT.Revenue_DATE, 
                        WAT.Service_Start_Date, 
                        WAT.Service_End_Date, 
                        WAT.Days, 
                        WAT.Meter_Number, 
                        SUM(WAT.Consumption_hcf) Consumption_hcf
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) WAT
                ON ELEC.revenue_date= WAT.revenue_date
                AND ELEC.location=WAT.location
                AND ELEC.Account_Name=WAT.Account_Name
            GROUP BY 1,2,3,4,5,6,7,8,9,10,11)A
        GROUP BY 1,2,3,4,5
        HAVING COUNT(*) >1;
                ''')
    result_join_A=cur.fetchall()
    conn.commit()
except psycopg2.errors.InFailedSqlTransaction as e:
    conn.rollback()
    print("Transaction rolled back:", e)
except Exception as e:
    # handle any other exceptions that might occur
    print("Error occurred:", e)
len(result_join_A)

801

In [26]:
result_join_A[1:10]

[('UPACA (SITE 5)',
  'MANHATTAN',
  'UPACA (SITE 5)',
  'BLD 01',
  datetime.datetime(2014, 7, 1, 0, 0),
  2),
 ('SOUTH BRONX AREA (SITE 402)',
  'BRONX',
  'SOUTH BRONX AREA (SITE 402)',
  'BLD 01',
  datetime.datetime(2013, 11, 1, 0, 0),
  2),
 ('UPACA (SITE 5)',
  'MANHATTAN',
  'UPACA (SITE 5)',
  'BLD 01',
  datetime.datetime(2014, 1, 1, 0, 0),
  3),
 ('BUSHWICK II CDA (GROUP E)',
  'BROOKLYN',
  'BUSHWICK II CDA (GROUP E)',
  'BLD 02',
  datetime.datetime(2013, 12, 1, 0, 0),
  2),
 ('TAPSCOTT STREET REHAB',
  'BROOKLYN',
  'TAPSCOTT STREET REHAB',
  'BLD 04',
  datetime.datetime(2014, 4, 1, 0, 0),
  2),
 ('BUSHWICK II CDA (GROUP E)',
  'BROOKLYN',
  'BUSHWICK II CDA (GROUP E)',
  'BLD 02',
  datetime.datetime(2014, 5, 1, 0, 0),
  2),
 ('BUSHWICK II CDA (GROUP E)',
  'BROOKLYN',
  'BUSHWICK II CDA (GROUP E)',
  'BLD 02',
  datetime.datetime(2013, 7, 1, 0, 0),
  2),
 ('STANTON STREET',
  'MANHATTAN',
  'STANTON STREET',
  'BLD 01',
  datetime.datetime(2014, 3, 1, 0, 0),
  2),
 ('S

In [27]:
#checking the reson behind the cartesian product
#CHECKING OF GETTING CARTESIAN PRODUCT AFTER LEFT JOIN

try:
    
    cur.execute('''
    SELECT
                ELEC.Development_Name, 
                ELEC.Borough, 
                ELEC.Account_Name,
                ELEC.location ,
                ELEC.Revenue_DATE, 
                ELEC.Service_Start_Date elec_Service_Start_Date, 
                ELEC.Service_End_Date elec_Service_End_Date, 
                ELEC.Days elec_Days, 
                WAT.Service_Start_Date water_Service_Start_Date, 
                WAT.Service_End_Date water_Service_End_Date, 
                WAT.Days Water_days, 
                SUM(ELEC.Consumption_KWH) elec_Consumption_KWH, 
                SUM(ELEC.KWH_Charges) elec_KWH_Charges, 
                SUM(ELEC.Consumption_KW) elec_Consumption_KW, 
                SUM(ELEC.KW_Charges) elec_KW_Charges, 
                SUM(WAT.Consumption_hcf) AS water_Consumption_hcf
                FROM(SELECT
                    ELEC.BILL_ID AS BILL_ID,
                    ELEC.Development_Name AS Development_Name, 
                    ELEC.Borough AS Borough, 
                    ELEC.Account_Name AS Account_Name,
                    ELEC.location AS "location" ,
                    ELEC.Revenue_DATE AS Revenue_DATE, 
                    ELEC.Service_Start_Date AS Service_Start_Date, 
                    ELEC.Service_End_Date AS Service_End_Date, 
                    ELEC.Days AS Days, 
                    ELEC.Meter_Number AS Meter_Number, 
                    SUM(ELEC.Consumption_KWH) AS Consumption_KWH, 
                    SUM(ELEC.KWH_Charges) AS KWH_Charges, 
                    SUM(ELEC.Consumption_KW) AS Consumption_KW, 
                    SUM(ELEC.KW_Charges) AS KW_Charges
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) ELEC
                Left JOIN (SELECT 
                        WAT.BILL_ID,
                        WAT.Development_Name, 
                        WAT.Borough, 
                        WAT.Account_Name,
                        WAT.location ,
                        WAT.Revenue_DATE, 
                        WAT.Service_Start_Date, 
                        WAT.Service_End_Date, 
                        WAT.Days, 
                        WAT.Meter_Number, 
                        SUM(WAT.Consumption_hcf) Consumption_hcf
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) WAT
                ON ELEC.revenue_date= WAT.revenue_date
                AND ELEC.location=WAT.location
                AND ELEC.Account_Name=WAT.Account_Name
            WHERE 1=1
            AND TRIM(ELEC.Development_Name)='UPACA (SITE 5)'
            AND TRIM(ELEC.Account_Name)='UPACA (SITE 5)'
            AND TRIM(ELEC.location)='BLD 01'
            AND ELEC.revenue_date='2014-07-01'
            GROUP BY 1,2,3,4,5,6,7,8,9,10,11
                ''')
    result_join_B=cur.fetchall()
    conn.commit()
except psycopg2.errors.InFailedSqlTransaction as e:
    conn.rollback()
    print("Transaction rolled back:", e)
except Exception as e:
    # handle any other exceptions that might occur
    print("Error occurred:", e)


In [29]:
result_join_B

[('UPACA (SITE 5)',
  'MANHATTAN',
  'UPACA (SITE 5)',
  'BLD 01',
  datetime.datetime(2014, 7, 1, 0, 0),
  datetime.datetime(2014, 6, 24, 0, 0),
  datetime.datetime(2014, 7, 24, 0, 0),
  30,
  datetime.datetime(2014, 6, 22, 0, 0),
  datetime.datetime(2014, 7, 22, 0, 0),
  30,
  Decimal('105096.0'),
  Decimal('6436.2'),
  Decimal('176.0'),
  Decimal('2087.4'),
  Decimal('22386')),
 ('UPACA (SITE 5)',
  'MANHATTAN',
  'UPACA (SITE 5)',
  'BLD 01',
  datetime.datetime(2014, 7, 1, 0, 0),
  datetime.datetime(2014, 6, 24, 0, 0),
  datetime.datetime(2014, 7, 24, 0, 0),
  30,
  datetime.datetime(2014, 6, 28, 0, 0),
  datetime.datetime(2014, 7, 29, 0, 0),
  31,
  Decimal('52548.0'),
  Decimal('3218.1'),
  Decimal('88.0'),
  Decimal('1043.7'),
  Decimal('0'))]

In [35]:
#there is difference in columns "Service_Start_Date" ,"Service_End_Date" , "days" in water_comsumption table,
#so for to handle this

#CHECKING OF GETTING CARTESIAN PRODUCT AFTER LEFT JOIN

try:
    
    cur.execute('''SELECT
    Development_Name,
    Borough,
    Account_Name,
    location,
    Revenue_DATE,
    COUNT(*)
    FROM (
    SELECT
                ELEC.Development_Name, 
                ELEC.Borough, 
                ELEC.Account_Name,
                ELEC.location ,
                ELEC.Revenue_DATE, 
                MIN(ELEC.Service_Start_Date) elec_Service_Start_Date, 
                MAX(ELEC.Service_End_Date) elec_Service_End_Date, 
                --ELEC.Days elec_Days, 
                MIN(WAT.Service_Start_Date) water_Service_Start_Date, 
                MAX(WAT.Service_End_Date) water_Service_End_Date, 
                --WAT.Days Water_days, 
                SUM(ELEC.Consumption_KWH) elec_Consumption_KWH, 
                SUM(ELEC.KWH_Charges) elec_KWH_Charges, 
                SUM(ELEC.Consumption_KW) elec_Consumption_KW, 
                SUM(ELEC.KW_Charges) elec_KW_Charges, 
                SUM(WAT.Consumption_hcf) AS water_Consumption_hcf
                FROM(SELECT
                    ELEC.BILL_ID AS BILL_ID,
                    ELEC.Development_Name AS Development_Name, 
                    ELEC.Borough AS Borough, 
                    ELEC.Account_Name AS Account_Name,
                    ELEC.location AS "location" ,
                    ELEC.Revenue_DATE AS Revenue_DATE, 
                    ELEC.Service_Start_Date AS Service_Start_Date, 
                    ELEC.Service_End_Date AS Service_End_Date, 
                    ELEC.Days AS Days, 
                    ELEC.Meter_Number AS Meter_Number, 
                    SUM(ELEC.Consumption_KWH) AS Consumption_KWH, 
                    SUM(ELEC.KWH_Charges) AS KWH_Charges, 
                    SUM(ELEC.Consumption_KW) AS Consumption_KW, 
                    SUM(ELEC.KW_Charges) AS KW_Charges
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) ELEC
                Left JOIN (SELECT 
                        WAT.BILL_ID,
                        WAT.Development_Name, 
                        WAT.Borough, 
                        WAT.Account_Name,
                        WAT.location ,
                        WAT.Revenue_DATE, 
                        WAT.Service_Start_Date, 
                        WAT.Service_End_Date, 
                        WAT.Days, 
                        WAT.Meter_Number, 
                        SUM(WAT.Consumption_hcf) Consumption_hcf
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8,9,10) WAT
                ON ELEC.revenue_date= WAT.revenue_date
                AND ELEC.location=WAT.location
                AND ELEC.Account_Name=WAT.Account_Name
            GROUP BY 1,2,3,4,5)A
        GROUP BY 1,2,3,4,5
        HAVING COUNT(*) >1;
                ''')
    result_join_C=cur.fetchall()
    conn.commit()
except psycopg2.errors.InFailedSqlTransaction as e:
    conn.rollback()
    print("Transaction rolled back:", e)
except Exception as e:
    # handle any other exceptions that might occur
    print("Error occurred:", e)



In [36]:
#Now we are not getting cartesian product in join
len(result_join_C)

0

In [37]:
#IPROVISING THE QUERY 
#AND CHECKING OF GETTING CARTESIAN PRODUCT AFTER IMPROVISING

try:
    
    cur.execute('''SELECT
    Development_Name,
    Borough,
    Account_Name,
    location,
    Revenue_DATE,
    COUNT(*)
    FROM (SELECT
                ELEC.Development_Name, 
                ELEC.Borough, 
                ELEC.Account_Name,
                ELEC.location ,
                ELEC.Revenue_DATE, 
                SUM(Elec_Meter_count) Elec_Meter_count,
                SUM(Water_Meter_count) Water_Meter_count,
                MIN(ELEC.Service_Start_Date) elec_Service_Start_Date, 
                MAX(ELEC.Service_End_Date) elec_Service_End_Date, 
                --ELEC.Days elec_Days, 
                MIN(WAT.Service_Start_Date) water_Service_Start_Date, 
                MAX(WAT.Service_End_Date) water_Service_End_Date, 
                --WAT.Days Water_days, 
                AVG(ELEC.Consumption_KWH) elec_Consumption_KWH, 
                AVG(ELEC.KWH_Charges) elec_KWH_Charges, 
                AVG(ELEC.Consumption_KW) elec_Consumption_KW, 
                AVG(ELEC.KW_Charges) elec_KW_Charges, 
                AVG(WAT.Consumption_hcf) AS water_Consumption_hcf
                FROM(SELECT
                    --ELEC.BILL_ID AS BILL_ID,
                    ELEC.Development_Name AS Development_Name, 
                    ELEC.Borough AS Borough, 
                    ELEC.Account_Name AS Account_Name,
                    ELEC.location AS "location" ,
                    ELEC.Revenue_DATE AS Revenue_DATE, 
                    ELEC.Service_Start_Date AS Service_Start_Date, 
                    ELEC.Service_End_Date AS Service_End_Date, 
                    ELEC.Days AS Days, 
                    COUNT(DISTINCT ELEC.Meter_Number) AS Elec_Meter_count,
                    AVG(ELEC.Consumption_KWH) AS Consumption_KWH, 
                    AVG(ELEC.KWH_Charges) AS KWH_Charges, 
                    AVG(ELEC.Consumption_KW) AS Consumption_KW, 
                    AVG(ELEC.KW_Charges) AS KW_Charges
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8) ELEC
                Left JOIN (SELECT 
                        --WAT.BILL_ID,
                        WAT.Development_Name, 
                        WAT.Borough, 
                        WAT.Account_Name,
                        WAT.location ,
                        WAT.Revenue_DATE, 
                        WAT.Service_Start_Date, 
                        WAT.Service_End_Date, 
                        WAT.Days, 
                        COUNT(DISTINCT WAT.Meter_Number) Water_Meter_count, 
                        AVG(WAT.Consumption_hcf) Consumption_hcf
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8) WAT
                ON ELEC.revenue_date= WAT.revenue_date
                AND ELEC.location=WAT.location
                AND ELEC.Account_Name=WAT.Account_Name
            GROUP BY 1,2,3,4,5)A
        GROUP BY 1,2,3,4,5
        HAVING COUNT(*) >1;
                ''')
    final_join=cur.fetchall()
    conn.commit()
except psycopg2.errors.InFailedSqlTransaction as e:
    conn.rollback()
    print("Transaction rolled back:", e)
except Exception as e:
    # handle any other exceptions that might occur
    print("Error occurred:", e)



In [38]:
final_join

[]

In [49]:
#Joining newyork_weather to the query
try:
    
    cur.execute(''' DROP TABLE IF EXISTS Electrical_Water_Consumption;
    CREATE TABLE Electrical_Water_Consumption AS SELECT
                ELEC.Development_Name, 
                ELEC.Borough, 
                ELEC.Account_Name,
                ELEC.location ,
                ELEC.Revenue_DATE, 
                AVG_TEMP,
                SUM(Elec_Meter_count) Elec_Meter_count,
                SUM(Water_Meter_count) Water_Meter_count,
                MIN(ELEC.Service_Start_Date) elec_Service_Start_Date, 
                MAX(ELEC.Service_End_Date) elec_Service_End_Date, 
                --ELEC.Days elec_Days, 
                MIN(WAT.Service_Start_Date) water_Service_Start_Date, 
                MAX(WAT.Service_End_Date) water_Service_End_Date, 
                --WAT.Days Water_days, 
                AVG(ELEC.Consumption_KWH) elec_Consumption_KWH, 
                AVG(ELEC.KWH_Charges) elec_KWH_Charges, 
                AVG(ELEC.Consumption_KW) elec_Consumption_KW, 
                AVG(ELEC.KW_Charges) elec_KW_Charges, 
                AVG(WAT.Consumption_hcf) AS water_Consumption_hcf
                FROM(SELECT
                    --ELEC.BILL_ID AS BILL_ID,
                    ELEC.Development_Name AS Development_Name, 
                    ELEC.Borough AS Borough, 
                    ELEC.Account_Name AS Account_Name,
                    ELEC.location AS "location" ,
                    ELEC.Revenue_DATE AS Revenue_DATE, 
                    ELEC.Service_Start_Date AS Service_Start_Date, 
                    ELEC.Service_End_Date AS Service_End_Date, 
                    ELEC.Days AS Days, 
                    COUNT(DISTINCT ELEC.Meter_Number) AS Elec_Meter_count,
                    AVG(ELEC.Consumption_KWH) AS Consumption_KWH, 
                    AVG(ELEC.KWH_Charges) AS KWH_Charges, 
                    AVG(ELEC.Consumption_KW) AS Consumption_KW, 
                    AVG(ELEC.KW_Charges) AS KW_Charges
                    FROM electric_consumption ELEC
                    GROUP BY 1,2,3,4,5,6,7,8) ELEC
                Left JOIN (SELECT 
                        --WAT.BILL_ID,
                        WAT.Development_Name, 
                        WAT.Borough, 
                        WAT.Account_Name,
                        WAT.location ,
                        WAT.Revenue_DATE, 
                        WAT.Service_Start_Date, 
                        WAT.Service_End_Date, 
                        WAT.Days, 
                        COUNT(DISTINCT WAT.Meter_Number) Water_Meter_count, 
                        AVG(WAT.Consumption_hcf) Consumption_hcf
                    FROM water_consumption WAT
                    GROUP BY 1,2,3,4,5,6,7,8) WAT
                ON ELEC.revenue_date= WAT.revenue_date
                AND ELEC.location=WAT.location
                AND ELEC.Account_Name=WAT.Account_Name
                
                inner JOIN (SELECT TO_CHAR("date", 'yyyy-mm-01')::date AS year_mnth,
                    AVG((temp_max+temp_min)/2) AS AVG_TEMP
                    
                    FROM  newyork_weather
                    GROUP BY 1
                )WTHER
                ON ELEC.revenue_date = WTHER.year_mnth
            GROUP BY 1,2,3,4,5,6;
                ''')
    
    conn.commit()
except psycopg2.errors.InFailedSqlTransaction as e:
    conn.rollback()
    print("Transaction rolled back:", e)
except Exception as e:
    # handle any other exceptions that might occur
    print("Error occurred:", e)

In [48]:
len(final_join)

161988

In [ ]:

cur.close()
conn.close()